In [21]:
import pandas as pd
import numpy as np
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [1]:
import zipfile
import os

# Path to your zip file
zip_path = "/content/archive.zip"

# Directory to extract to
extract_dir = "extracted_files"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Extracted to: {extract_dir}")


✅ Extracted to: extracted_files


In [20]:
pip install pandas scikit-learn LibRecommender surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469530 sha256=896eb56eaedc4dd0df8997ea072f4c5e1185bb2b078b051fbbc7bd17ef6a76c3
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from libreco.data import DatasetPure, random_split
from libreco.algorithms import NCF

# Load your course dataset
courses = pd.read_csv("/content/extracted_files/coursea_data.csv")

# Clean and encode course titles as item IDs
courses['item'] = courses.index.astype(int)  # unique numeric ID for each course


**Simulated Intern-Course Ratings **

In [15]:
np.random.seed(42)
n_users = 50
n_items = len(courses)
ratings = []

for user in range(1, n_users + 1):
    selected = np.random.choice(courses['item'], 10, replace=False)
    for item in selected:
        ratings.append([user, item, np.random.randint(3, 6)])  # Ratings: 3 to 5

ratings_df = pd.DataFrame(ratings, columns=['user', 'item', 'rating'])


**Train the SVD Model**

In [22]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user', 'item', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
svd_model = SVD()
svd_model.fit(trainset)


** Content-Based Filtering (TF-IDF on Metadata)**

In [26]:
# Combine metadata into single content field
courses['content'] = (
    courses['course_title'].astype(str) + ' ' +
    courses['course_organization'].astype(str) + ' ' +
    courses['course_Certificate_type'].astype(str) + ' ' +
    courses['course_difficulty'].astype(str)
)

# TF-IDF + Cosine Similarity
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(courses['content'])
content_sim = cosine_similarity(tfidf_matrix)

# Map item ID to index
item_to_index = {item: idx for idx, item in enumerate(courses['item'])}


**Hybrid Recommendation Function**

In [27]:
def hybrid_recommend(user_id, top_n=5, alpha=0.7):
    rated_items = ratings_df[ratings_df['user'] == user_id]['item'].tolist()
    all_items = courses['item'].tolist()
    candidates = [i for i in all_items if i not in rated_items]

    recommendations = []

    for item in candidates:
        # Collaborative score from SVD
        collab_score = svd_model.predict(user_id, item).est

        # Content score: average similarity to rated items
        sim_scores = []
        for rated in rated_items:
            if item in item_to_index and rated in item_to_index:
                sim_scores.append(content_sim[item_to_index[item]][item_to_index[rated]])
        content_score = np.mean(sim_scores) if sim_scores else 0

        # Hybrid score
        final_score = alpha * collab_score + (1 - alpha) * content_score
        recommendations.append((item, final_score))

    # Top-N Recommendations
    top_recs = sorted(recommendations, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\n🎓 Top {top_n} Recommended Courses for Intern {user_id}:\n")
    for item_id, score in top_recs:
        row = courses[courses['item'] == item_id].iloc[0]
        print(f"- {row['course_title']} ({row['course_organization']}) | Difficulty: {row['course_difficulty']} | Score: {score:.2f}")


**Run the Recommender**

In [28]:
hybrid_recommend(user_id=10, top_n=5, alpha=0.7)



🎓 Top 5 Recommended Courses for Intern 10:

- Introduction to Business Analytics: Communicating with Data (University of Illinois at Urbana-Champaign) | Difficulty: Intermediate | Score: 2.99
- Methods and Statistics in Social Sciences (University of Amsterdam) | Difficulty: Beginner | Score: 2.98
- The Science of Gastronomy (The Hong Kong University of Science and Technology) | Difficulty: Mixed | Score: 2.97
- High Performance Collaboration: Leadership, Teamwork, and Negotiation (Northwestern University) | Difficulty: Mixed | Score: 2.97
- Anatomy (University of Michigan) | Difficulty: Beginner | Score: 2.97
